In [8]:
import os

#os.environ["HF_DATASETS_CACHE"] = "/domino/datasets/local/SAS-R/"
os.environ["TRANSFORMERS_CACHE"] =  "/mnt/data/CodeGen"

In [9]:
from transformers.utils import TRANSFORMERS_CACHE
TRANSFORMERS_CACHE

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/mnt/data/CodeGen'

In [10]:
import torch

print(torch.cuda.is_available())
print(torch.__version__)

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import transformers

True
2.0.1+cu118


In [11]:
model_name = 'codellama/CodeLlama-7b-Instruct-hf'
tokenizer = AutoTokenizer.from_pretrained(model_name, cache_dir=TRANSFORMERS_CACHE)
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             cache_dir=TRANSFORMERS_CACHE,
                                             torch_dtype=torch.bfloat16,
                                             device_map="auto",
                                            )


Loading checkpoint shards: 100%|██████████| 2/2 [00:44<00:00, 22.07s/it]


In [12]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation",
                model=model,
                tokenizer= tokenizer
               )

In [13]:
code="""
/* Load the input dataset */
proc import out=cars_data
    datafile="/mnt/data/mtcars.csv"
    dbms=csv
    replace;
    getnames=YES;
run;

/* Sort the input dataset by displacement */
proc sort data=cars_data;
      by disp;
run;

/* Perform the linear regression */
proc reg data=cars_data;
      model mpg = disp / noprint;
      output out=output_data predicted=mpg_predicted;
run;

/* Print the regression results */
/* In this case a scatter plot with regression line best fit */
proc sgplot data=output_data;
      scatter x=disp y=mpg / markerattrs=(symbol=circlefilled) name='scatter';
      series x=disp y=mpg_predicted / lineattrs=(color=blue) name='regression';
      keylegend 'scatter' 'regression';
run;
"""

In [28]:
convert= "You are a code assistant.Please write R code using statistics and plotting libraries that will reproduce the results of the SAS code that follows. Output the result in markdown \n"
thisPrompt="{}\n{} ```r ".format(convert, code)

In [47]:
import random

random.seed(42)

sequences = pipe(
    thisPrompt,
    do_sample=True,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=500
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [48]:
for seq in sequences:
    output_text = seq['generated_text']
    output_text = output_text.replace(thisPrompt,"")
    print(output_text)


library(dplyr)
library(MASS)
library(sasstats)

c <-read.csv("path_of_your_data")

cars_sorted <-c %>% 
  select(disp, mpg) %>% 
  arrange(desc(disp))

call <- lm(mpg ~ disp, data=cars_sorted)
summary(call)
prediction5<- predict(call,newdata=cars_sorted)
output_data<-data.frame(
  'disp'= c(cars_sorted$disp),
 'mpg'=c(cars_sorted$mpg),
  
 'mpg_predicted' = as.numeric(prediction5),
)
library(ggplot2)
b<-ggplot(output_data,aes(x=disp,y=mpg))+
    geom_point() +
    ylab("mpg") +
    xlab("disp") +
    geom_line(aes(y = mpg_predicted,x = disp),color="blue") +
    geom_point(color = "red")+
    geom_line(aes(y = prediction5,x = disp),color = "green")        
b
 ```

**Answer**

The SAS code is provided above.The plot generated using R  code and output is below:

![alt](https://jasonwinn.gitbooks.io/hands_on_statistics_and_data_visualization/content/4/sas/sasviz_graph.png)



In [81]:
convert= "You are a helpful polite code assistant.Please write R code using statistics for example stats and plotting libraries for example ggplot for the task that follows. Output the result in markdown \n Task : "
task = "Give me code to load data from a csv and perform linear regression once the data has been loaded"
thisPrompt="{}\n{} ```r ".format(convert, task)

sequences = pipe(
    thisPrompt,
    do_sample=True,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    max_new_tokens=500
)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [82]:
for seq in sequences:
    output_text = seq['generated_text']
    output_text = output_text.replace(thisPrompt,"")
    print(output_text)



#load your data
mydata <- read.csv("file.csv") 
str(mydata)


#plot and view your data
ggplot(mydata,aes(x=x,y=y))+geom_point()


#Perform a linear regression of your chosen X and Y
summary(lm(mydata$y~mydata$x)) 


```


